<a href="https://colab.research.google.com/github/Dererkzzz/ECE1512-2023F-ProjectRepo-Longqian-Zhang-Yuxuan-Liu/blob/main/task2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow.compat.v2 as tf #
from typing import Union
from tensorflow.keras.layers import *
from keras.preprocessing.image import ImageDataGenerator
#from keras.utils.np_utils import to_categorical
import tensorflow.keras.utils as utils
import keras
import numpy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


tf.enable_v2_behavior()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
target_directory = r'/content/drive/My Drive/ECE_1512_ProjectA/mhist_dataset/images'
os.chdir(target_directory)
image_table=pd.read_csv('/content/drive/My Drive/ECE_1512_ProjectA/mhist_dataset/annotations.csv')




train dataset


In [ ]:
train_set = image_table[image_table['Partition'] == 'train']
print("Shape of the 'train_set' DataFrame:")
print(train_set.shape)

Shape of the 'train_set' DataFrame:
(2175, 4)


test dataset

In [ ]:
test_set = image_table[image_table['Partition'] == 'test']
print("Shape of the 'test_set' DataFrame:")
print(test_set.shape)

Shape of the 'test_set' DataFrame:
(977, 4)


In [ ]:
image_dir = '/content/drive/My Drive//ECE_1512_ProjectA/mhist_dataset/images'

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1/255.0,      # 将像素值缩放到0到1之间
    shear_range=-0.1,      # 错切变换范围
    rotation_range=15,    # 旋转范围，最多旋转15度
    horizontal_flip=True, # 水平翻转
    vertical_flip=True,    # 垂直翻转
    zoom_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2
)

test_datagen = ImageDataGenerator(
    rescale=1/255.0  # 将像素值缩放到0到1之间
)


create dataset generator

In [ ]:
# 创建训练数据生成器
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_set,             # 从数据框中读取训练数据
    directory=image_dir,             # 图像所在的目录路径
    x_col="Image Name",              # 图像文件名列
    y_col="Majority Vote Label",     # 类别标签列
    batch_size=32,                   # 批次大小
    seed=50,                         # 随机种子，可使生成的数据可重复
    shuffle=True,                    # 是否随机洗牌数据
    interpolation='lanczos',        # 图像大小调整插值方法
    class_mode="categorical",        # 多分类问题
    target_size=(224, 224)           # 目标图像大小
)


Found 2175 validated image filenames belonging to 2 classes.


In [ ]:
test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_set,             # 从数据框中读取训练数据
    directory=image_dir,             # 图像所在的目录路径
    x_col="Image Name",              # 图像文件名列
    y_col="Majority Vote Label",     # 类别标签列
    batch_size=32,                   # 批次大小
    seed=50,                         # 随机种子，可使生成的数据可重复
    shuffle=False,                    # 是否随机洗牌数据
    interpolation='lanczos',        # 图像大小调整插值方法
    class_mode="categorical",        # 多分类问题
    target_size=(224, 224)           # 目标图像大小
)


Found 977 validated image filenames belonging to 2 classes.


Create ResNet50v2 model

In [ ]:
resnet = tf.keras.applications.ResNet50V2(
    include_top = False,  # 不包括顶层（全连接层）
    weights="imagenet",  # 使用预训练权重
    input_tensor=None,  # 没有指定输入张量，将在模型内部创建
    input_shape=(224, 224, 3),  # 输入图像的形状为 224x224 像素，RGB通道
    pooling='avg'  # 使用全局平均池化层
)

94668760/94668760 [==============================] - 5s 0us/step


In [ ]:
# 获取 ResNet 模型
resnet = tf.keras.applications.ResNet50V2(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=(224, 224, 3),
    pooling='avg'
)

# 定义要冻结的层的数量
freeze_layers = 170

# 冻结模型的前 freeze_layers 层
for layer in resnet.layers[:freeze_layers]:
    layer.trainable = False

#输出已冻结层的数量
print(f"frozen {freeze_layers} layer")

frozen 170 layer


In [ ]:
#检查模型层的可训练性质和名称
for idx, layer in enumerate(resnet.layers, 1):
    print(f"layer {idx} {layer.trainable},  {layer.name[:5]}")

layer 1 False,  input
layer 2 False,  conv1
layer 3 False,  conv1
layer 4 False,  pool1
layer 5 False,  pool1
layer 6 False,  conv2
layer 7 False,  conv2
layer 8 False,  conv2
layer 9 False,  conv2
layer 10 False,  conv2
layer 11 False,  conv2
layer 12 False,  conv2
layer 13 False,  conv2
layer 14 False,  conv2
layer 15 False,  conv2
layer 16 False,  conv2
layer 17 False,  conv2
layer 18 False,  conv2
layer 19 False,  conv2
layer 20 False,  conv2
layer 21 False,  conv2
layer 22 False,  conv2
layer 23 False,  conv2
layer 24 False,  conv2
layer 25 False,  conv2
layer 26 False,  conv2
layer 27 False,  conv2
layer 28 False,  conv2
layer 29 False,  conv2
layer 30 False,  conv2
layer 31 False,  conv2
layer 32 False,  conv2
layer 33 False,  conv2
layer 34 False,  conv2
layer 35 False,  conv2
layer 36 False,  conv2
layer 37 False,  conv2
layer 38 False,  max_p
layer 39 False,  conv2
layer 40 False,  conv2
layer 41 False,  conv3
layer 42 False,  conv3
layer 43 False,  conv3
layer 44 False,  con

In [ ]:
# 创建全连接输出层
output_layer = tf.keras.layers.Dense(units=2, activation='sigmoid')

# 创建教师模型
teacher_res = tf.keras.Sequential([resnet, output_layer])
teacher_res.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50v2 (Functional)     (None, 2048)              23564800  
                                                                 
 dense (Dense)               (None, 2)                 4098      
                                                                 
Total params: 23568898 (89.91 MB)
Trainable params: 7883778 (30.07 MB)
Non-trainable params: 15685120 (59.83 MB)
_________________________________________________________________


Create mobilenet model

In [ ]:
mobilenet = tf.keras.applications.MobileNetV2(
    include_top=False,  # 不包括顶层（全连接层）
    weights="imagenet",  # 使用预训练权重
    input_tensor=None,  # 没有指定输入张量，将在模型内部创建
    input_shape=(224, 224, 3),  # 输入图像的形状为 224x224 像素，RGB通道
    pooling='max'
)

9406464/9406464 [==============================] - 1s 0us/step


In [ ]:

# freeze_layers_start = 'block_11'  # 指定要从哪一层开始冻结
# for layer in mobilenet.layers:
#     if layer.name.startswith(freeze_layers_start):
#         break
#     layer.trainable = False

# # 输出已冻结层的数量
# num_trainable_layers = len([layer for layer in mobilenet.layers if layer.trainable])
# print(f"frozen {len(mobilenet.layers) - num_trainable_layers} layer")
# Making the first 120 layers trainable
count=0
for layers in mobilenet.layers:
  count+=1
  if count> 130:
    layers.trainable=True
  # if(layers.name[0:8]== 'block_13' ):
  #   continue
  else:
    layers.trainable=False
print(count)


155


In [ ]:
count=0
for layers in mobilenet.layers:
  print(layers.trainable, layers.name[0:8], count)
  count+=1

False input_3 0
False Conv1 1
False bn_Conv1 2
False Conv1_re 3
False expanded 4
False expanded 5
False expanded 6
False expanded 7
False expanded 8
False block_1_ 9
False block_1_ 10
False block_1_ 11
False block_1_ 12
False block_1_ 13
False block_1_ 14
False block_1_ 15
False block_1_ 16
False block_1_ 17
False block_2_ 18
False block_2_ 19
False block_2_ 20
False block_2_ 21
False block_2_ 22
False block_2_ 23
False block_2_ 24
False block_2_ 25
False block_2_ 26
False block_3_ 27
False block_3_ 28
False block_3_ 29
False block_3_ 30
False block_3_ 31
False block_3_ 32
False block_3_ 33
False block_3_ 34
False block_3_ 35
False block_4_ 36
False block_4_ 37
False block_4_ 38
False block_4_ 39
False block_4_ 40
False block_4_ 41
False block_4_ 42
False block_4_ 43
False block_4_ 44
False block_5_ 45
False block_5_ 46
False block_5_ 47
False block_5_ 48
False block_5_ 49
False block_5_ 50
False block_5_ 51
False block_5_ 52
False block_5_ 53
False block_6_ 54
False block_6_ 55
False 

In [ ]:
output_layer = tf.keras.layers.Dense(units=2, activation='sigmoid')

# 创建学生模型
student_mobile = tf.keras.Sequential([mobilenet, output_layer])

Training

# Teacher loss function

In [ ]:

def compute_teacher_loss(images, labels):
  """Compute subclass knowledge distillation teacher loss for given images
     and labels.

  Args:
    images: Tensor representing a batch of images.
    labels: Tensor representing a batch of labels.

  Returns:
    Scalar loss Tensor.
  """
  subclass_logits = teacher_res(images, training=True)

  # Compute cross-entropy loss for subclasses.

  # your code start from here for step 3
  cross_entropy_loss_value = tf.keras.losses.CategoricalCrossentropy(from_logits=True)(labels, subclass_logits)


  return cross_entropy_loss_value

# Student loss function

In [ ]:
##@test {"output": "ignore"}

# Hyperparameters for distillation (need to be tuned).
ALPHA = 0.5 # task balance between cross-entropy and distillation loss
DISTILLATION_TEMPERATURE = 4. #temperature hyperparameter

def distillation_loss(teacher_logits: tf.Tensor, student_logits: tf.Tensor,
                      temperature: Union[float, tf.Tensor]):
  """Compute distillation loss.

  This function computes cross entropy between softened logits and softened
  targets. The resulting loss is scaled by the squared temperature so that
  the gradient magnitude remains approximately constant as the temperature is
  changed. For reference, see Hinton et al., 2014, "Distilling the knowledge in
  a neural network."

  Args:
    teacher_logits: A Tensor of logits provided by the teacher.
    student_logits: A Tensor of logits provided by the student, of the same
      shape as `teacher_logits`.
    temperature: Temperature to use for distillation.

  Returns:
    A scalar Tensor containing the distillation loss.
  """
 # your code start from here for step 3
  soft_targets = tf.nn.softmax(teacher_logits/temperature)

  return tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(
          soft_targets, student_logits / temperature)) * temperature ** 2

def compute_student_loss(images, labels, ALPHA, DISTILLATION_TEMPERATURE):
  """Compute subclass knowledge distillation student loss for given images
     and labels.

  Args:
    images: Tensor representing a batch of images.
    labels: Tensor representing a batch of labels.

  Returns:
    Scalar loss Tensor.
  """
  student_subclass_logits = student_mobile(images, training=True) # fc model

  # Compute subclass distillation loss between student subclass logits and
  # softened teacher subclass targets probabilities.

  # your code start from here for step 3

  teacher_subclass_logits = teacher_res(images, training=False) # cnn_model
  distillation_loss_value = distillation_loss(teacher_subclass_logits, student_subclass_logits,
                      DISTILLATION_TEMPERATURE)

  # Compute cross-entropy loss with hard targets.

  # your code start from here for step 3

  cross_entropy_loss_value = tf.keras.losses.CategoricalCrossentropy(from_logits=True)(labels, student_subclass_logits)

  return ALPHA * distillation_loss_value + (1-ALPHA)* cross_entropy_loss_value

# Train and evaluation


In [ ]:

def compute_num_correct(model, images, labels):
  """Compute number of correctly classified images in a batch.

  Args:
    model: Instance of tf.keras.Model.
    images: Tensor representing a batch of images.
    labels: Tensor representing a batch of labels.

  Returns:
    Number of correctly classified images.
  """
  class_logits = model(images, training=False)
  return tf.reduce_sum(
      tf.cast(tf.math.equal(tf.argmax(class_logits, -1), tf.argmax(labels, -1)),
              tf.float32)), tf.argmax(class_logits, -1), tf.argmax(labels, -1)


def train_and_evaluate_teacher(model, compute_loss_fn, num_epochs=10):
    """Train and evaluate the teacher model.

    Args:
        model: Instance of tf.keras.Model.
        compute_loss_fn: A function to compute training loss from images and labels.
        num_epochs: Number of training epochs.
    """

    # Set the optimizer for training.
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)

    # Training loop for the specified number of epochs.
    for epoch in range(1, num_epochs + 1):
        print('Epoch {}: '.format(epoch), end='')

        # Reset the training data generator.
        train_generator.reset()

        # Loop through batches for training.
        for _ in range(68):
            images, labels = train_generator.next()

            # Calculate loss and gradients.
            with tf.GradientTape() as tape:
                loss_value = compute_loss_fn(images, labels)

            grads = tape.gradient(loss_value, model.trainable_variables)

            # Apply gradients to update model parameters.
            optimizer.apply_gradients(zip(grads, model.trainable_variables))

        # Evaluate the model after each epoch.
        num_correct = 0
        num_total = 0

        for _ in range(31):
            images, labels = test_generator.next()
            num_total += len(labels)
            num_correct += compute_num_correct(model, images, labels)[0]

        print("Class accuracy: " + '{:.2f}%'.format(num_correct / num_total * 100))

    # Fine-tuning phase.
    print('Fine-tuning')

    # Set the optimizer for fine-tuning.
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.00001)

    # Get the specific layer for fine-tuning.
    # m = teacher_res.get_layer('resnet50v2')
    # count = 0

    # Fine-tune specific layers.
    # for layers in m.layers:
    #     count += 1
    #     if count == 152 or layers.name[0:5] == 'conv4':
    #         layers.trainable = True
    #     else:
    #         continue

    m=teacher_res.get_layer('resnet50v2')
    count = 0
    for layers in m.layers:
      count+=1
      print(layers.trainable, layers.name[0:5], count)


    # Training loop for fine-tuning.
    for epoch in range(1, 25 + 1):
        print('Epoch {}: '.format(epoch), end='')

        # Reset the training data generator.
        train_generator.reset()

        # Loop through batches for fine-tuning.
        for _ in range(68):
            images, labels = train_generator.next()

            # Calculate loss and gradients.
            with tf.GradientTape() as tape:
                loss_value = compute_loss_fn(images, labels)

            grads = tape.gradient(loss_value, model.trainable_variables)

            # Apply gradients to update model parameters.
            optimizer.apply_gradients(zip(grads, model.trainable_variables))

        # Evaluate the model during fine-tuning.
        num_correct = 0
        num_total = 0

        for _ in range(31):
            images, labels = test_generator.next()
            num_total += len(labels)
            num_correct += compute_num_correct(model, images, labels)[0]

        print("Class accuracy: " + '{:.2f}%'.format(num_correct / num_total * 100))





In [ ]:
def train_and_evaluate_kd(model, compute_loss_fn, ALPHA=0.8, DISTILLATION_TEMPERATURE=8.):
  """Perform training and evaluation for a given model.

  Args:
    model: Instance of tf.keras.Model.
    compute_loss_fn: A function that computes the training loss given the
      images, and labels.
  """

  # your code start from here for step 4
  optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

  for epoch in range(1,11):
    # Run training.
    print('Epoch {}: '.format(epoch), end='')
    train_generator.reset()
    for _ in range(68):
      images, labels=train_generator.next()
      with tf.GradientTape() as tape:
         # your code start from here for step 4

        loss_value = compute_loss_fn(images,labels, ALPHA, DISTILLATION_TEMPERATURE)

      grads = tape.gradient(loss_value, model.trainable_variables)
      optimizer.apply_gradients(zip(grads, model.trainable_variables))

    # Run evaluation.
    num_correct = 0
    num_total = 0
    for _ in range(31):
      # your code start from here for step 4
      images, labels=test_generator.next()
      num_total+= len(labels)
      num_correct += compute_num_correct(model,images,labels)[0]
    print("Class_accuracy: " + '{:.2f}%'.format(
        num_correct / num_total * 100))

  #fine tuning
  print('fine tuning')
  optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)

  m=student_mobile.get_layer('mobilenetv2_1.00_224')

  count=0
  for layers in m.layers:
    count+=1
    print(layers.trainable, layers.name[0:8], count)

  for epoch in range(1,26):
  # Run training.
    print('Epoch {}: '.format(epoch), end='')
    train_generator.reset()
    for _ in range(68):
      images, labels=train_generator.next()
      with tf.GradientTape() as tape:
        # your code start from here for step 4

        loss_value = compute_loss_fn(images,labels, ALPHA, DISTILLATION_TEMPERATURE)

      grads = tape.gradient(loss_value, model.trainable_variables)
      optimizer.apply_gradients(zip(grads, model.trainable_variables))

    # Run evaluation.
    num_correct = 0
    num_total = 0
    for _ in range(31):
      # your code start from here for step 4
      images, labels=test_generator.next()
      num_total+= len(labels)
      num_correct += compute_num_correct(model,images,labels)[0]
    print("Class_accuracy: " + '{:.2f}%'.format(
        num_correct / num_total * 100))

In [ ]:
def train_and_evaluate_no_kd(model, compute_loss_fn):
  """Perform training and evaluation for a given model.

  Args:
    model: Instance of tf.keras.Model.
    compute_loss_fn: A function that computes the training loss given the
      images, and labels.
  """

  # your code start from here for step 4
  optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

  for epoch in range(10):
    # Run training.
    print('Epoch {}: '.format(epoch), end='')
    train_generator.reset()
    for _ in range(68):
      images, labels=train_generator.next()
      with tf.GradientTape() as tape:
         # your code start from here for step 4

        loss_value = compute_loss_fn(images,labels)

      grads = tape.gradient(loss_value, model.trainable_variables)
      optimizer.apply_gradients(zip(grads, model.trainable_variables))

          # Run evaluation.
    num_correct = 0
    num_total = 0
    for _ in range(31):
      # your code start from here for step 4
      images, labels=test_generator.next()
      num_total+= len(labels)
      num_correct += compute_num_correct(model,images,labels)[0]
    print("Class_accuracy: " + '{:.2f}%'.format(
        num_correct / num_total * 100))

  #fine tuning
  print('fine tuning')
  optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)

  m=student_mobile.get_layer('mobilenetv2_1.00_224')
  count=0
  for layers in m.layers:
    count+=1
    print(layers.trainable, layers.name[0:8], count)

  for epoch in range(25):
  # Run training.
    print('Epoch {}: '.format(epoch), end='')
    train_generator.reset()
    for _ in range(68):
      images, labels=train_generator.next()
      with tf.GradientTape() as tape:
        # your code start from here for step 4

        loss_value = compute_loss_fn(images,labels)

      grads = tape.gradient(loss_value, model.trainable_variables)
      optimizer.apply_gradients(zip(grads, model.trainable_variables))

    # Run evaluation.
    num_correct = 0
    num_total = 0
    for _ in range(31):
      # your code start from here for step 4
      images, labels=test_generator.next()
      num_total+= len(labels)
      num_correct += compute_num_correct(model,images,labels)[0]
    print("Class_accuracy: " + '{:.2f}%'.format(
        num_correct / num_total * 100))


In [ ]:
def compute_plain_cross_entropy_loss_mobile(images, labels):
  """Compute plain loss for given images and labels.

  For fair comparison and convenience, this function also performs a
  LogSumExp over subclasses, but does not perform subclass distillation.

  Args:
    images: Tensor representing a batch of images.
    labels: Tensor representing a batch of labels.

  Returns:
    Scalar loss Tensor.
  """
  # your code start from here for step 7

  student_subclass_logits = student_mobile(images, training=True)
  cross_entropy_loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)(labels, student_subclass_logits)

  return cross_entropy_loss

training

In [ ]:
train_and_evaluate_teacher(teacher_res,compute_teacher_loss)

Epoch 1: 

/usr/local/lib/python3.10/dist-packages/keras/src/backend.py:5577: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


Class accuracy: 78.20%
Epoch 2: Class accuracy: 79.53%
Epoch 3: Class accuracy: 81.47%
Epoch 4: Class accuracy: 81.27%
Epoch 5: Class accuracy: 79.63%
Epoch 6: Class accuracy: 80.45%
Epoch 7: Class accuracy: 81.37%
Epoch 8: Class accuracy: 81.06%
Epoch 9: Class accuracy: 82.19%
Epoch 10: Class accuracy: 81.88%
Fine-tuning
False input 1
False conv1 2
False conv1 3
False pool1 4
False pool1 5
False conv2 6
False conv2 7
False conv2 8
False conv2 9
False conv2 10
False conv2 11
False conv2 12
False conv2 13
False conv2 14
False conv2 15
False conv2 16
False conv2 17
False conv2 18
False conv2 19
False conv2 20
False conv2 21
False conv2 22
False conv2 23
False conv2 24
False conv2 25
False conv2 26
False conv2 27
False conv2 28
False conv2 29
False conv2 30
False conv2 31
False conv2 32
False conv2 33
False conv2 34
False conv2 35
False conv2 36
False conv2 37
False max_p 38
False conv2 39
False conv2 40
False conv3 41
False conv3 42
False conv3 43
False conv3 44
False conv3 45
False conv

In [ ]:
model_path = '/content/drive/My Drive/ECE_1512_ProjectA/mhist_dataset/teacher_res_new'
tf.keras.models.save_model(teacher_res, model_path)
loaded_model = tf.keras.models.load_model(model_path)

In [ ]:
train_and_evaluate_no_kd(student_mobile, compute_plain_cross_entropy_loss_mobile)

In [ ]:
model_path = '/content/drive/My Drive/ECE_1512_ProjectA/mhist_dataset/mobilenet_no_kd_new'
tf.keras.models.save_model(student_mobile, model_path)
loaded_model = tf.keras.models.load_model(model_path)

In [ ]:
model_path = '/content/drive/My Drive/ECE_1512_ProjectA/mhist_dataset/student_mobile_new'
tf.keras.models.save_model(student_mobile, model_path)
loaded_model = tf.keras.models.load_model(model_path)

In [ ]:
train_and_evaluate_kd(student_mobile, compute_student_loss)

Epoch 1: 

/usr/local/lib/python3.10/dist-packages/keras/src/backend.py:5577: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


Class_accuracy: 68.47%
Epoch 2: Class_accuracy: 71.65%
Epoch 3: Class_accuracy: 72.57%
Epoch 4: Class_accuracy: 77.28%
Epoch 5: Class_accuracy: 71.75%
Epoch 6: Class_accuracy: 74.72%
Epoch 7: Class_accuracy: 70.93%
Epoch 8: Class_accuracy: 69.91%
Epoch 9: Class_accuracy: 80.35%
Epoch 10: Class_accuracy: 75.95%
fine tuning
False input_3 1
False Conv1 2
False bn_Conv1 3
False Conv1_re 4
False expanded 5
False expanded 6
False expanded 7
False expanded 8
False expanded 9
False block_1_ 10
False block_1_ 11
False block_1_ 12
False block_1_ 13
False block_1_ 14
False block_1_ 15
False block_1_ 16
False block_1_ 17
False block_1_ 18
False block_2_ 19
False block_2_ 20
False block_2_ 21
False block_2_ 22
False block_2_ 23
False block_2_ 24
False block_2_ 25
False block_2_ 26
False block_2_ 27
False block_3_ 28
False block_3_ 29
False block_3_ 30
False block_3_ 31
False block_3_ 32
False block_3_ 33
False block_3_ 34
False block_3_ 35
False block_3_ 36
False block_4_ 37
False block_4_ 38
False

In [ ]:
model_path = '/content/drive/My Drive/ECE_1512_ProjectA/mhist_dataset/student_mobile_kd'
tf.keras.models.save_model(student_mobile, model_path)
loaded_model = tf.keras.models.load_model(model_path)